# Iterational Solution

Стратегия такая:
1. Сортируем поезда по длине их маршрутов. Если длины их маршрутов совпадают, то сортируем по значению минимальной пропускной способности по убыванию.
2. Берем самый длинный поезд. Выписываем все маршруты, которые ему доступны (столбцы матрицы `Capacity`: то есть все пары $(a,b)$, где $a,b$ -- города из маршрута этого поезда).
3. Выбираем самый длинный маршрут.
4. Дальше надо понять, сколько вагонов будут отправлены по этому маршруту. Это число -- это минимум между тем, сколько надо отправить по этому маршруту вагонов (элемент матрицы `Needs`) и минимальной пропускной способностью на этом маршруте.
5. Выбрав число вагонов, переписываем матрицу `Needs` и пропускную способность, вычитая выбранные вагоны.
6. Так делаем, пока для всех маршрутов не будет записано, сколько вагонов отправляется.

In [73]:
import pandas as pd
import numpy as np
from itertools import permutations, combinations

In [28]:
dataset = pd.read_json('dataset.json')
dataset.head()

,stations,full_timetable
0,"{'Златоуст (1)': ['0', '38', '38', '25', '29',...","{'853': {'route': ['1', '3', '2'], 'free_carri..."
1,"{'Златоуст (1)': ['0', '28', '24', '28', '8', ...","{'896': {'route': ['1', '3', '2'], 'free_carri..."
2,"{'Златоуст (1)': ['0', '25', '15', '6', '15', ...","{'309': {'route': ['1', '3', '2'], 'free_carri..."
3,"{'Златоуст (1)': ['0', '32', '30', '13', '4', ...","{'800': {'route': ['2', '3', '6', '5'], 'free_..."
4,"{'Златоуст (1)': ['0', '10', '38', '16', '1', ...","{'893': {'route': ['1', '3', '6', '5', '7'], '..."


In [249]:
data_to_see = dataset.iloc[0]
full_timetable = data_to_see['full_timetable']
stations = data_to_see['stations']

In [250]:
Needs = np.array([list(stations.values())], dtype=np.int32)[0]
Needs

array([[ 0, 38, 38, 25, 29,  7, 10],
       [26,  0,  7, 34, 20, 27, 35],
       [15,  5,  0, 27, 16, 31, 24],
       [12, 39,  1,  0,  2, 14, 20],
       [38, 38,  1, 28,  0, 33, 14],
       [21, 27, 24,  9,  1,  0,  3],
       [ 9,  3, 23, 25, 32, 37,  0]], dtype=int32)

In [251]:
routes = [list(map(lambda x: int(x), list(full_timetable.values())[i]['route'])) for i in range(len(full_timetable.keys()))]
free_carriages = [list(map(lambda x: int(x), list(full_timetable.values())[i]['free_carriage'])) for i in range(len(full_timetable.keys()))]

trains = list(full_timetable.keys())
n_trains = len(trains)
trains_encodding = {i: train for i, train in enumerate(trains)}
trains_codes = range(n_trains)

paths = list(permutations(cities, 2))
paths_encodding = {path: i for i, path in enumerate(paths)}

cities = list(map(lambda x: int(x[-2:-1]), stations.keys()))

In [252]:
print(f"routes are {routes}")

routes are [[1, 3, 2], [1, 3, 2], [1, 3, 6, 5, 4], [1, 3, 6, 5, 7], [1, 3, 6, 5, 7], [2, 3, 1], [2, 3, 6, 5], [2, 3, 6, 5, 7], [2, 3, 6, 5, 7], [2, 3, 6, 7], [2, 3, 6, 7], [3, 2, 4], [3, 2, 4], [3, 2, 4, 5], [4, 2], [4, 2, 3], [4, 2, 3], [4, 2, 3], [4, 5], [4, 5, 6, 7], [4, 5, 6, 7], [4, 5, 7], [4, 5, 7], [5, 4], [5, 4, 2, 3, 1], [5, 4, 2, 3, 1], [5, 6, 3, 1], [5, 6, 3, 2], [5, 6, 3, 2], [5, 6, 3, 2], [5, 6, 7], [5, 6, 7], [5, 7], [5, 7], [5, 7], [7, 5], [7, 5, 4], [7, 5, 4], [7, 5, 6, 3, 1], [7, 5, 6, 3, 2], [7, 6, 3, 1], [7, 6, 5], [7, 6, 5, 4]]


In [253]:
Capacity = np.zeros((len(trains), len(paths)))
#print(len(paths), len(trains), Capacity.shape)
for i, train in enumerate(trains):
    route = tuple(map(lambda x: int(x), full_timetable[train]['route']))
    free_carriage = list(map(lambda x: int(x), full_timetable[train]['free_carriage']))
    for j, path in enumerate(paths):
        #path is in route
        #that is, each element of path is in route
        #AND index of path[0] < index of path[1] in route
        if path[0] in route and path[1] in route:
            ind_of_0 = route.index(path[0])
            ind_of_1 = route.index(path[1])
            if ind_of_0 < ind_of_1:
                #print(f"path is = {path}\t path is = {route}")
                #print(f"free_carriage is {free_carriage}")
                value = min(free_carriage[ind_of_0 : ind_of_1])
                #print(f"value is {value}")
                Capacity[i,j] = value
Capacity

array([[21., 21.,  0., ...,  0.,  0.,  0.],
       [10., 34.,  0., ...,  0.,  0.,  0.],
       [ 0., 33.,  3., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0., 16.],
       [ 0.,  0.,  0., ...,  0.,  1.,  1.],
       [ 0.,  0.,  0., ...,  5., 28., 39.]])

## Sort routes and free carriage

In [271]:
sortings = sorted(zip(routes, free_carriages, trains_codes), key=lambda x: (len(x[0]), -min(x[1])))
routes, free_carriages, trains_codes = list(zip(*sortings))
print(f"trains are {[trains_encodding[code] for code in trains_codes]}")
print(f"routes are {routes}")
print(f"free carriages are {free_carriages}")

trains are ['156', '222', '417', '719', '328', '371', '223', '448', '973', '692', '432', '853', '621', '177', '801', '743', '275', '967', '500', '942', '300', '324', '200', '700', '232', '774', '399', '332', '617', '140', '331', '444', '323', '722', '405', '182', '658', '901', '930', '883', '563', '251', '643']
routes are ([5, 7], [5, 7], [5, 7], [7, 5], [4, 2], [4, 5], [5, 4], [5, 6, 7], [7, 5, 4], [3, 2, 4], [4, 2, 3], [1, 3, 2], [5, 6, 7], [4, 5, 7], [1, 3, 2], [2, 3, 1], [4, 2, 3], [3, 2, 4], [4, 5, 7], [4, 2, 3], [7, 5, 4], [7, 6, 5], [4, 5, 6, 7], [7, 6, 3, 1], [5, 6, 3, 2], [5, 6, 3, 2], [2, 3, 6, 5], [2, 3, 6, 7], [2, 3, 6, 7], [3, 2, 4, 5], [7, 6, 5, 4], [5, 6, 3, 2], [4, 5, 6, 7], [5, 6, 3, 1], [5, 4, 2, 3, 1], [1, 3, 6, 5, 7], [2, 3, 6, 5, 7], [5, 4, 2, 3, 1], [2, 3, 6, 5, 7], [7, 5, 6, 3, 1], [1, 3, 6, 5, 4], [7, 5, 6, 3, 2], [1, 3, 6, 5, 7])
free carriages are ([36], [25], [22], [21], [18], [12], [8], [37, 37], [24, 36], [26, 23], [23, 28], [21, 21], [15, 34], [21, 14], [3

In [272]:
Cars = [None] * n_trains

In [273]:
for i, train in enumerate(trains_codes):
    train_route = routes[i]
    route_len = len(train_route)
    train_free_carriage = np.array(free_carriages[i])
    Cars[i] = {}
    for j, start in enumerate(train_route):
        for end in train_route[:j:-1]:
            start_ind = train_route.index(start)
            end_ind = train_route.index(end)
            path = (start, end)
            need = Needs[path[0]-1, path[1]-1]
            train_capacity = min(train_free_carriage[start_ind : end_ind])
            cars = min(need, train_capacity)
            Cars[i][path] = cars
            Needs[path[0]-1, path[1]-1] -= cars
            train_free_carriage[start_ind : end_ind] -= cars

In [274]:
Cars[40]

{(1, 4): 3,
 (1, 5): 0,
 (1, 6): 0,
 (1, 3): 0,
 (3, 4): 0,
 (3, 5): 0,
 (3, 6): 0,
 (6, 4): 0,
 (6, 5): 0,
 (5, 4): 0}

In [275]:
free_carriages[40], routes[40]

([33, 20, 35, 3], [1, 3, 6, 5, 4])

In [259]:
Needs

array([[ 0,  7,  0, 22,  8,  5,  0],
       [ 5,  0,  0,  9,  0,  0,  0],
       [ 0,  0,  0,  0,  3, 10, 20],
       [12,  0,  0,  0,  0,  0,  0],
       [ 1,  6,  0,  0,  0,  0,  0],
       [ 7,  3,  0,  9,  0,  0,  0],
       [ 0,  0, 11,  0,  0, 25,  0]], dtype=int32)

In [260]:
np.array([list(stations.values())], dtype=np.int32)[0]

array([[ 0, 38, 38, 25, 29,  7, 10],
       [26,  0,  7, 34, 20, 27, 35],
       [15,  5,  0, 27, 16, 31, 24],
       [12, 39,  1,  0,  2, 14, 20],
       [38, 38,  1, 28,  0, 33, 14],
       [21, 27, 24,  9,  1,  0,  3],
       [ 9,  3, 23, 25, 32, 37,  0]], dtype=int32)